## Задача 2
Загрузите подготовленные в Задаче 1 данные в pd.DataFrame. Выполните анализ рекламных кампаний на канале tiktokglobal_int за предложенный период времени. Определите, какие кампании можно считать эффективными или перспективными, а какие — нет.
Критерий эффективности кампании: пользователи, привлеченные этой кампанией, на 7-й день жизни после регистрации (то есть через 168 часов) приносят доход, который превышает 18% от рекламных затрат (ROAS 7 > 18%). Отсчет дней жизни ведется с момента регистрации пользователя интервалами по 24 часа.

## План:

- Загрузить CSV в pandas.DataFrame, проверить типы данных, привести к корректным, если необходимо;
- Отфильтровать строки, где media_source == "tiktokglobal_int";
- Сгруппировать данные по campaign, чтобы сложить все доходы и затраты за период:
  - sum(costs)
  - sum(revenue) — общий доход за анализируемый период (используем как total_revenue_7)
- Посчитать ROAS 7 по формуле ROAS7 = costs / total_revenue_7;
- Создать колонку status:
  - эффективная — если roas_7 > 0.18
  - неэффективная — иначе
- Вывести итоговую таблицу — campaign | costs | total_revenue_7 | roas_7 | status.

In [5]:
# Загрузить CSV в pandas.DataFrame, проверить типы данных

import pandas as pd

df = pd.read_csv("marketing_summary_by_date_channel_campaign.csv")
print(df.dtypes)


date             object
media_source     object
campaign         object
installs          int64
revenue         float64
costs           float64
impressions       int64
clicks            int64
dtype: object


In [3]:
# Изментиь тип данных Data с object на datetime

df['date'] = pd.to_datetime(df['date'])
print(df.dtypes)

date            datetime64[ns]
media_source            object
campaign                object
installs                 int64
revenue                float64
costs                  float64
impressions              int64
clicks                   int64
dtype: object


In [6]:
# Фильтрация только по каналу tiktokglobal_int

tiktok_df = df[df['media_source'] == 'tiktokglobal_int']
tiktok_df.head()

,date,media_source,campaign,installs,revenue,costs,impressions,clicks
0,2024-03-01,tiktokglobal_int,tt_campaign_1,254,0.000000,373.682987,55620,2201
1,2024-03-01,tiktokglobal_int,tt_campaign_4,93,0.000000,130.908847,27650,675
2,2024-03-02,tiktokglobal_int,tt_campaign_1,218,11.823100,370.283288,85610,1470
3,2024-03-02,tiktokglobal_int,tt_campaign_4,60,2.350400,123.301634,17787,494
4,2024-03-03,tiktokglobal_int,tt_campaign_1,405,58.466801,373.219923,74222,2512


In [8]:
# Группируем по кампании и суммируем траты и доход

grouped = (
    tiktok_df
      .groupby('campaign', as_index=False)[['costs', 'revenue']]
      .sum()
)

In [9]:
# Переименуем 'revenue' в 'total_revenue_7' (доход за анализируемый период)

grouped = grouped.rename(columns={'revenue': 'total_revenue_7'})

In [10]:
# ROAS_7 = total_revenue_7 / costs

grouped['roas_7'] = (grouped['total_revenue_7'] / grouped['costs']).fillna(0).round(4)

In [16]:
# Статус по порогу 18% (0.18)

grouped['status'] = grouped['roas_7'].gt(0.18).map({True: 'эффективная', False: 'неэффективная'})
grouped.to_csv('tiktok_campaign_eval.csv', index=False)
print(grouped.head(10))

        campaign        costs  total_revenue_7  roas_7         status
0  tt_campaign_1  9331.254887      2191.308934  0.2348    эффективная
1  tt_campaign_2  2879.353299       440.453019  0.1530  неэффективная
2  tt_campaign_3  4893.326561      1316.011202  0.2689    эффективная
3  tt_campaign_4   892.362465       400.746598  0.4491    эффективная


## Выводы по результатам анализа

- На канале tiktokglobal_int за анализируемый период были запущены 4 кампании.  
- По критерию ROAS_7 > 0.18:
  - 3 кампании признаны эффективными,  
  - 1 кампания признана неэффективной.  
- Наиболее успешные кампании:
  - tt_campaign_4 — ROAS_7 ≈ 0.45
  - tt_campaign_3 — ROAS_7 ≈ 0.27
  - tt_campaign_1 — ROAS_7 ≈ 0.23
- Кампания tt_campaign_2 показала ROAS_7 ≈ 0.15 и не достигла порогового значения эффективности.

Рекомендации:
- Продолжить и масштабировать кампании с высоким ROAS_7 (tt_campaign_4, tt_campaign_3, tt_campaign_1).
- Проанализировать причины низкой окупаемости tt_campaign_2 (креативы, аудитории, ставки) и при отсутствии улучшений — остановить кампанию.
